Here we perform a check on DE and FR datasets to confirm choice of loss function for our data.

In [19]:
import os
import pandas as pd
import numpy as np
import subprocess
from utils.helper import extract_metrics_from_output

# Test for Informer

In [25]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE", "RSE"]
country = dataset[:2]

log_dir = f"logs/loss_choice"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [27]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"

informer_results = []

with open(log_file_path, "w") as log_file:
    for loss in losses:

      log_file.write(f"\n=== Starting experiments for loss function: {loss} ===\n")

      for pred_len in pred_lens:
        model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

        # Arguments for the command
        command = f"""
        python {script_path} \
          --random_seed 2021 \
          --is_training 1 \
          --root_path "{data_path}" \
          --data_path "{dataset}" \
          --model_id {model_id} \
          --model "{model}" \
          --data "custom" \
          --features M \
          --seq_len {seq_len} \
          --label_len 5 \
          --pred_len {pred_len} \
          --e_layers 2 \
          --d_layers 1 \
          --factor 5 \
          --enc_in 5 \
          --dec_in 5 \
          --c_out 5 \
          --des 'Exp' \
          --train_epochs 10 \
          --patience 3 \
          --overlapping_windows \
          --loss_fnc "{loss}" \
          --itr 2 --batch_size 32 --learning_rate "{lr}"
        """

        # Run the command and capture the output
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Capture the output in real-time
        output = []
        for line in process.stdout:
            output.append(line)
            print(line, end='')  # Print in the .ipynb cell
            log_file.write(line)  # Write to the log file

        # Wait for the process to complete
        process.wait()

        # Extract metrics from the captured output
        mse, rmse, mae, rse = extract_metrics_from_output(output)

        # Log the extracted metrics
        log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
        log_file.write(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, RSE: {rse}\n")

        # Append the results to the informer_results list
        informer_results.append({
            'Loss_function': loss,
            'Pred_len': pred_len,
            'MSE': mse,
            'RMSE': np.sqrt(mse),
            'MAE': mae,
            'RSE': np.sqrt(mse) / mae
        })

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/cs-hu/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, seq_len=96, label_len=5, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type3', pct_start=0.3, use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [ ]:
# Convert the collected data into a pandas DataFrame
informer_df = pd.DataFrame(informer_results)

# Set multi-index 
informer_df.set_index(['Loss_function', 'Pred_len'], inplace=True)
informer_df = informer_df.round(4)